### 1) 라이브러리 임포트

In [2]:
import os, random, numpy as np, pandas as pd, torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          get_scheduler)          
from torch.optim import AdamW                   
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

import tensorflow as tf

/Users/shinjoohwan/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"사용 디바이스: {device}")

사용 디바이스: cpu


### 2) 데이터 불러오기

In [5]:
train_path = "NLPtrain.csv"
test_path  = "NLPtest.csv"

train_df = pd.read_csv(train_path)
test_df  = pd.read_csv(test_path)

### 3) Label Encoding

In [10]:
le = LabelEncoder() # LabelEncoder 객체 생성(0 ~ n_classes-1 사이의 정수로 매핑)
train_df["label_id"] = le.fit_transform(train_df["label"]) # fit: trian_df["label"] 열을 스캔해 고유 레이블 목록(le.classes_)을 학습 / transform: 학습된 순서에 따라 각 레이블을 정수로 변환
NUM_LABELS = len(le.classes_) # le.classes: 식별된 고유 레이블 배열 -> 클래스 개수 계산

### 4) Train / Valudation (라벨 분포 유지)

In [13]:
train_df, val_df = train_test_split(
    train_df,
    test_size=0.2,
    stratify=train_df["label_id"],
    random_state=SEED,
)

### 5) 토크나이저 & 동적 MAX_LEN 산출

In [16]:
MODEL_NAME = "bert-base-multilingual-cased" # 다국어 대소문자 구분 BERT
tokenizer  = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
# AutoTokenizer는 모델 이름만 주면 자동으로 맞춤 토크나이저 클래스 불러옴

def calc_dynamic_max_len(texts, percentile=95, hard_cap=512): # texts: 학습, 추론에 사용할 문장 리스트 / percentile: 몇 퍼센트 지점까지 커버할지 결정 / hard_cap: BERT 구조적 한계(512 토큰) 넘지 않도록 상한선 
    # tokenizer.encode(): 문장 t를 토큰 ID 시퀀스로 변환
    # add_special_tokens=TRUE: 토크나이저가 모델에 필요한 '특수 토큰'을 자동으로 앞.뒤에 붙여서 인코딩하도록 지시하는 매개변수
    lengths = [len(tokenizer.encode(t, add_special_tokens=True))
               for t in tqdm(texts, desc="길이 측정")]
    dyn_len = int(np.percentile(lengths, percentile)) # lenghts 리스트(각 문장의 토큰 수 모음)의 분포를 통계적으로 살펴보고, 지정한 분위값에 해당하는 길이를 구하는 과정
    return min(dyn_len, hard_cap)

MAX_LEN = calc_dynamic_max_len(train_df["data"], percentile=95)
print(f"동적으로 산출된 MAX_LEN = {MAX_LEN}")

BATCH_SIZE = 32
EPOCHS     = 20

길이 측정: 100%|███████████████████████████| 200/200 [00:00<00:00, 26250.49it/s]

동적으로 산출된 MAX_LEN = 79


### 6) PyTorch Dataset 정의

In [18]:
class TextDataset(Dataset): # 텍스트를 토큰화해 텐서로 변환, (학습 단계라면) 레이블도 함께 반환하도록 설계
    def __init__(self, df, training=True):
        self.texts     = df["data"].tolist() # 문장 리스트를 미리 추출해 메모리 상에 보관
        self.labels    = df["label_id"].tolist() if training else None # # 학습 모드일 때만 레이블 리스트를 저장
        self.training  = training # 학습 모드 여부
        
    def __len__(self):  
        return len(self.texts)
        
    def __getitem__(self, idx):
        # 텍스트 토큰화
        enc = tokenizer(
            self.texts[idx],
            truncation=True, # MAX_LEN보다 길면 자름
            padding="max_length", # 빈 칸을 PAD 토큰으로 채움
            max_length=MAX_LEN, # 앞서 계산한 동적 최대 길이
            return_tensors="pt", # PyTorch 텐서 형태로 반환
        )
        # tokenizer에서 return_tensors='pt'를 쓰면 첫 차원(batch)=1인 텐서가 생성
        # squeeze(0)으로 불필요한 차원을 제거해 MAX_LEN(1D) 텐서로 변환
        item = {k: v.squeeze(0) for k, v in enc.items()}
        if self.training:
            item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long) # 레이블을 torch.long형 텐서로 변환 후 item 딕셔너리에 추가
        return item

train_ds = TextDataset(train_df)
val_ds   = TextDataset(val_df)
test_ds  = TextDataset(test_df, training=False)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE)

### 7) 모델, 옵티마이저, 스케줄러

In [22]:
# AutoModelForSequenceClassification: 허깅페이스가 "시퀀스 분류용" 헤드를 자동으로 붙인 래퍼 클래스
# 사전 학습 가중치를 불러오면서, 마지막 분류 레이의 출력 뉴런 수를 NUM_LABELS개로 교체
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS
).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5) # model.parameters(): 학습 대상 파라미터 전부 전달
total_steps  = len(train_loader) * EPOCHS # 전체 학습 과정에서 optimizer가 파라미터를 업데이트하는 총 횟수
lr_scheduler = get_scheduler(
    name="linear", # 선형 감소 스케줄 선택 / 훈련이 진행됨에 따라 학습률이 초기값 -> 0으로 직선형으로 떨어짐
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * total_steps), # 전체 스텝의 10%를 워밍업으로 지정. 워밍업 동안 학습률이 0 -> 초기값까지 점진적으로 상승
    num_training_steps=total_steps,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 8) 학습 & 검증(Early-Stopping)

In [24]:
best_acc = 0.0 # 최고 검증 정확도
patience = 4 # Early Stopping 허용 횟수
pat_cnt =  0 # 최근 성능 개선 없었던 연속 epoch 수

for epoch in range(1, EPOCHS + 1):
    # ----- Train -----
    model.train()
    train_loss = 0.0 # epoch별 평균 Loss 계산용 누적 변수
    for batch in tqdm(train_loader, desc=f"[Epoch {epoch}] Train", leave=False):  # train_loader에서 배치를 하나씩 뽑아 온다
        batch = {k: v.to(device) for k, v in batch.items()} # 데이터를 디바이스로 이동
        outputs = model(**batch) # 순전파 /  **: 딕셔너리 언패킹 - 딕셔너리의 각 키 - 값 쌍을 키워드 인자로 풀어서 함수에 전달(키워드 인자: 파이썬 함수에 "인자 이름 = 값" 형태로 전달되는 매개변수)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # 그래디언트 클리핑: 역전파로 계산된 param.grad의 L2 노름이 1.0보다 크면 비율을 맞춰 축소 -> 너무 큰 그래디언트가 파라미터를  과하게 움직이는 것을 방지하여 학습 안정성 확보
        optimizer.step() # 클리핑된 그래디언트를 이용해 가중치를 1번 갱신
        lr_scheduler.step() # 스케줄러가 현재 스텝 번호에 맞춰 학습률을 선형 감소. 워밍업 등 설정한 전략에 따라 업데이트
        train_loss += loss.item() # tensor.item(): PyTorch 텐서 안에 값이 단 하나만 있을 때 그 값을 파이썬 스칼라(숫자 타입)로 꺼내 주는 메서드

    # ----- Validation -----
    model.eval()
    val_loss, preds, labels = 0.0, [], []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"[Epoch {epoch}] Val  ", leave=False): 
            batch = {k: v.to(device) for k, v in batch.items()}
            labels.extend(batch["labels"].cpu().numpy()) # 이번 배치의 labels 텐서를 다시 CPU로 옮겨 numpy 배열로 변환한 뒤, labels 리스트에 이어 붙여 전체 검증 데이터의 정답 라벨을 모은다
            outputs = model(**batch) # 로짓과 loss 반환
            val_loss += outputs.loss.item()
            preds.extend(torch.argmax(outputs.logits, 1).cpu().numpy()) # 로짓에서 argmax(가장 큰 점수 -> 예측 클래스)를 구한 뒤 리스트에 이어 붙여 전체 검증 데이터의 모든 예측을 모은다

    val_acc =. accuracy_score(labels, preds) # accuracy_score: 정확도 계산 함수
    print(f"Epoch {epoch} | Train {train_loss/len(train_loader):.4f} | "
          f"Val {val_loss/len(val_loader):.4f} | Acc {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        # 가중치만 저장
        torch.save(model.state_dict(), "best_model.h5") # model.state_dict(): 모델의 모든 학습 가능한 파라미터(가중치, 바이어스)를 파이썬 dict 형태로 반환(키: 레이어 이름 / 값: torch.Tensor(실제 수치 데이터))
        # 모델 객체 통째 저장
        # torch.save(model, "best_model_full.h5")
        print(f"모델 저장 (acc={best_acc:.4f})")
        pat_cnt = 0
    else:
        pat_cnt += 1
        if pat_cnt >= patience:
            print("Early stopping")
            break

print(f"최고 검증 정확도: {best_acc:.4f}")

Epoch 1 | Train 0.7001 | Val 0.6703 | Acc 0.6275
모델 저장 (acc=0.6275)


Epoch 2 | Train 0.6156 | Val 0.5433 | Acc 0.7843
모델 저장 (acc=0.7843)


Epoch 3 | Train 0.5036 | Val 0.4231 | Acc 0.8235
모델 저장 (acc=0.8235)


Epoch 4 | Train 0.2865 | Val 0.4888 | Acc 0.7647


Epoch 5 | Train 0.1622 | Val 0.3906 | Acc 0.8627
모델 저장 (acc=0.8627)


Epoch 6 | Train 0.0852 | Val 0.5299 | Acc 0.8431


Epoch 7 | Train 0.0344 | Val 0.4936 | Acc 0.8627


Epoch 8 | Train 0.0201 | Val 0.6788 | Acc 0.8235


Epoch 9 | Train 0.0067 | Val 0.6360 | Acc 0.8235
Early stopping
최고 검증 정확도: 0.8627


### 9) 테스트 예측 & CSV 저장

In [27]:
# best_model.h5 파일에서 state-dict(파라미터 텐서들의 딕셔너리)를 읽어 온다
model.load_state_dict(torch.load("best_model.h5", map_location=device))
model.eval()

# when 모델 객체 전부 다운했을 때 load 코드
# model_full = torch.load("best_model_full.pt", map_location=device)
# model.eval()

all_preds = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predict", leave=False):
        batch = {k: v.to(device) for k, v in batch.items()}
        logits = model(**batch).logits
        all_preds.extend(torch.argmax(logits, 1).cpu().numpy()) # 각 샘플에서 가장 큰 로짓 인덱스(=에측 클래스 ID)를 추출 -> numpy 배열로 변환해 all_preds에 이어 붙임

test_df["predict"] = le.inverse_transform(all_preds) # 학습 단계에서 사용한 LabelEncoder (le) 역변환 / 정수 ID -> 원본 문자 라벨
test_df.to_csv("test_with_pred.csv", index=False)
print("test_with_pred.csv 저장 완료")

test_with_pred.csv 저장 완료
